In [ ]:
import pandas as pd
import polars as pl
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

### Doing experiment for a random sample first, then extend to a for loop (Please change your file path and do the samplings in another jupyter notebook)

In [ ]:
testing_dataset = pd.read_parquet('/kaggle/input/helios-random-sampling-datasets/helios_random_samplings/sample_0.parquet')
testing_dataset.head()

In [ ]:
futures_cols = [c for c in testing_dataset.columns if c.startswith('futures_')]

### First filter: We only select the features have above 0.6 avg_sig_corr for the overall dataset.

In [ ]:
potential_features_df = pd.read_csv('/kaggle/input/final-corr/final_result_corr_2.csv') #
potential_features = potential_features_df.loc[
    potential_features_df['avg_sig_corr'] >= 0.6,
    'climate_variable'
].tolist()
print(len(potential_features))

In [ ]:
necessary_cols = ['crop_name','country_name','date_on_month']

In [ ]:
full_cols = necessary_cols + futures_cols + potential_features
print(len(full_cols))

### Load the CFCS backtest functions

In [ ]:
def compute_partial_correlations(df,by=['crop_name','country_name','date_on_month']):
    ## df: must be a dataframe with non-zero rows and at least two columns 
    ## each with name starting with climate_risk and futures
    
    ## by: groupby keyword value to partition df to compute partial
    ## correlations of each group
    ## default to groupings shown in Helios sample notebook

    def _climate_futures_corr_table(df,climate_risk_columns,futures_columns):
        ## compute correlations of each climate-futures variable pair
        ## Note: dataframe must contain at least two non-null pairs
        ## to produce a non-null correlation
        
        corr_matrix = df[climate_risk_columns+futures_columns]\
                      .corr(method='pearson',min_periods=2,numeric_only=True)
        ## corr() auto drop nan values before computing
        ## number pairs must be at least two for computation
        ## according to formula
        corr_table = corr_matrix.loc[climate_risk_columns,futures_columns]\
        .rename_axis(index='climate_variable',columns='futures_variable')\
        .stack().reset_index(name='correlation')
        ## drop nan correlations in stack operation
        return corr_table.round(5)

    climate_risk_columns = [c for c in df.columns if c.startswith('climate_risk')]
    futures_columns = [c for c in df.columns if c.startswith('futures')]
    df_default = pd.DataFrame([],columns=['correlation'])
    
    if len(climate_risk_columns)==0:
        print('input dataframe must have at least one column with name starting with climate_risk')
        return df_default
    
    if len(futures_columns)==0:
        print('input dataframe must have at least one column with name starting with futures')
        return df_default
    
    if by==None:
        corr_tables = _climate_futures_corr_table(df,climate_risk_columns,futures_columns)
    else:
        try:
            corr_tables = df.groupby(by=by).apply(_climate_futures_corr_table,\
                                                  climate_risk_columns,\
                                                  futures_columns,\
                                                  include_groups=False
                                                 )
            corr_tables = corr_tables\
                          .reset_index(level=len(corr_tables.index.levels)-1,drop=True)\
                          .reset_index()
            ## compute and combine the correlation table for each group
        except KeyError:
            print('illegal by values')
            return df_default

    return corr_tables

In [ ]:
def cfcs(df):
    """
    Calculate the Climate-Futures Correlation Score (CFCS) for leaderboard ranking.
    
    CFCS = (0.5 × Avg_Sig_Corr_Score) + (0.3 × Max_Corr_Score) + (0.2 × Sig_Count_Score)

    Input dataframe must have correlation column for computation
    """

    # Remove null correlations
    valid_corrs = df["correlation"].dropna()
    
    if len(valid_corrs) == 0:
        return {'cfcs_score': 0.0, 'error': 'No valid correlations'}
    
    # Calculate base metrics
    abs_corrs = valid_corrs.abs()
    max_abs_corr = abs_corrs.max()
    significant_corrs = abs_corrs[abs_corrs >= 0.5]
    significant_count = len(significant_corrs)
    total_count = len(valid_corrs)
    
    # Calculate component scores - ONLY average significant correlations
    if significant_count > 0:
        avg_sig_corr = significant_corrs.mean()
        avg_sig_score = min(100, avg_sig_corr * 100)  # Cap at 100 when avg sig reaches 1.0
    else:
        avg_sig_corr = 0.0
        avg_sig_score = 0.0
    
    max_corr_score = min(100, max_abs_corr * 100)  # Cap at 100 when max reaches 1.0
    sig_count_score = (significant_count / total_count) * 100  # Percentage
    
    # Composite score: Focus more on quality of significant correlations
    cfcs = (0.5 * avg_sig_score) + (0.3 * max_corr_score) + (0.2 * sig_count_score)
    scoreboard = {'cfcs_score': cfcs, 'avg_sig_score': avg_sig_score,\
                  'max_corr_score': max_corr_score,\
                  'sig_count_score': sig_count_score
                 }
    print(f'{round(sig_count_score,2)}% of all correlations are significant')
    print(f'Average significant correlation is {round(avg_sig_corr,3)}')
    print(f'highest absolute correlation found is {round(max_abs_corr,3)}')
    print(f'final CFCS score is {round(cfcs,2)}')
    return scoreboard

In [ ]:
def sigcorr_report(df, features='climate_variable', sig_level=0.5):
    '''
        Generate a CFCS sub scores report for each feature
        with significant correlations.
        
        Input dataframe must contain a correlation column and feature columns.
    
        Returns a DataFrame with columns:
        ['climate_variable', 'avg_sig_corr','max_sig_corr','sig_corr_count','sig_corr_ratio(%)']
    '''
    
    # Absolute correlation
    df['correlation_abs'] = df.correlation.abs()
    
    try:
        # Mask correlations below threshold
        df.loc[df['correlation_abs'] < sig_level, ['correlation_abs']] = np.nan
        
        # Group by feature
        reportdf = df.groupby(features).agg({
            'correlation_abs': ['mean','max','count'],
            'correlation': 'count'
        })
        
    except TypeError:
        print('sig_level must be a number between 0 and 1')
        return None
    except KeyError:
        print('illegal features values')
        return None
    
    # Flatten multi-level columns
    reportdf.columns = ['avg_sig_corr','max_sig_corr','sig_corr_count','total_corr_count']
    
    # Compute ratio
    reportdf['sig_corr_ratio(%)'] = 100 * reportdf['sig_corr_count'] / reportdf['total_corr_count']
    
    # Keep only rows with avg_sig_corr not null
    reportdf = reportdf[reportdf['avg_sig_corr'].notnull()]\
                       .loc[:, ['avg_sig_corr','max_sig_corr','sig_corr_count','sig_corr_ratio(%)']]\
                       .round(3)
    
    # Add the feature name as a column
    reportdf[features] = reportdf.index
    reportdf = reportdf.reset_index(drop=True)  # optional: reset index
    
    # Reorder columns so feature name is first
    reportdf = reportdf[[features, 'avg_sig_corr','max_sig_corr','sig_corr_count','sig_corr_ratio(%)']]
    
    return reportdf

### For loop, and get the features are performing consistently in these 20 parquet files, by sorting avg_sig_corr (18 features) with listing out the first 220 and sig_count_count (2 features) by listing out the first 100 

#### At the end of the subset, because of I missed some features, ended up we just have 15 features in our subset (final_submission.ipynb)

In [ ]:
import os

out_dir = '/kaggle/working/sampling_result/'
os.makedirs(out_dir, exist_ok=True)

In [ ]:
storage_set = None

for i in range(20):
    testings = pd.read_csv(f'/kaggle/input/helios-samplings-result/result_{i}')
    top_30_columns = (
        testings
        .sort_values(by='avg_sig_corr', ascending=False)
        .head(220)['climate_variable']
        .tolist()
    )

    current_set = set(top_30_columns)

    if storage_set is None:
        storage_set = current_set
    else:
        storage_set = storage_set.intersection(current_set)

storage_list = list(storage_set)

In [ ]:
print(len(storage_list))

In [ ]:
storage_list

### This is the avg_sig_corr features selected for final_submission:
'climate_risk_excess_precip_max_240d_cos_lag_7d',
'climate_risk_excess_precip_max_240d_ssqrt_above_3_std_lag_7d','climate_risk_excess_precip_max_240d_above_3_std_lag_14d',
'climate_risk_excess_precip_max_240d_above_3_std_lag_30d',
'climate_risk_drought_ma_90d_lag_7d',
'climate_risk_drought_ma_120d_tangent',
'climate_risk_drought_ma_120d_cos',
'climate_risk_drought_ma_120d_cos_lag_7d',
'climate_risk_drought_ma_90d_cos_lag_30d',
'climate_risk_drought_ma_120d_cos_above_1_std',
'climate_risk_drought_ma_120d_cos_above_2_std',
'climate_risk_drought_ma_60d_cos_above_2_std_lag_30d',
'climate_risk_drought_ma_120d_cos_above_3_std_lag_7d','climate_risk_coldwave_max_60d_thresh_mag_lag_30d',
'climate_risk_coldwave_max_60d_thresh_mag_above_1_std_lag_30d'